# EOEPCA Resource Discovery Validation and Usage Notebook

## Setup

In [6]:
import os
import requests
import json
from pathlib import Path

from owslib.csw import CatalogueServiceWeb
from owslib.ogcapi.records import Records
from owslib.opensearch import OpenSearch
from owslib.fes import And, Or, PropertyIsEqualTo, PropertyIsGreaterThanOrEqualTo, PropertyIsLessThanOrEqualTo, PropertyIsLike, BBox, SortBy, SortProperty
from geolinks import sniff_link
import folium

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [7]:
load_eoepca_state()

In [8]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'

print(f"Resource Discovery URL: {resource_discovery_domain}")

Resource Discovery URL: https://resource-catalogue.test.eoepca.org


## Validate Resource Discovery Endpoints

In [9]:
endpoints = [
    ("Landing Page", resource_discovery_domain),
    ("Swagger UI", f"{resource_discovery_domain}/openapi?f=html"),
    ("Collections", f"{resource_discovery_domain}/collections"),
    ("Conformance", f"{resource_discovery_domain}/conformance"),
    ("CSW GetCapabilities", f"{resource_discovery_domain}/csw?service=CSW&version=2.0.2&request=GetCapabilities"),
    ("STAC", f"{resource_discovery_domain}/stac")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

Landing Page (https://resource-catalogue.test.eoepca.org): 200
Swagger UI (https://resource-catalogue.test.eoepca.org/openapi?f=html): 200
Collections (https://resource-catalogue.test.eoepca.org/collections): 200
Conformance (https://resource-catalogue.test.eoepca.org/conformance): 200
CSW GetCapabilities (https://resource-catalogue.test.eoepca.org/csw?service=CSW&version=2.0.2&request=GetCapabilities): 200
STAC (https://resource-catalogue.test.eoepca.org/stac): 200


## Ingesting a Sample Record using STAC

In [23]:
sample_record_path = Path("example-item.json") # You will see this very simple sample file locally, feel free to experiment with it.
sample_collection = 'metadata:main'

ingest_url = f"{resource_discovery_domain}/collections/{sample_collection}/items"

with open(sample_record_path, 'r') as file:
    record = file.read()
record_json = json.loads(record)
record_id = record_json['id']
headers = {
    "Content-Type": "application/geo+json",
    "Accept": "application/json",
}
response = requests.post(ingest_url, headers=headers, data=record)

print(f"POST {ingest_url} - {response.status_code}")

POST https://resource-catalogue.test.eoepca.org/collections/metadata:main/items - 400


### Verify Record Ingestion

In [24]:
metadata_collection_url = f"{resource_discovery_domain}/collections/metadata:main/items"
response = requests.get(metadata_collection_url)

items = response.json().get("features", [])

for item in items:
    if item.get("id", "") == record_id:
        print("✅ Sample record successfully ingested and discoverable!")
        break

print(f'View the item in the catalogue: {resource_discovery_domain}/collections/{sample_collection}/items/{record_id}')

✅ Sample record successfully ingested and discoverable!
View the item in the catalogue: https://resource-catalogue.test.eoepca.org/collections/metadata:main/items/example-item


# CSW

In [25]:
system_catalogue_endpoint = f'{resource_discovery_domain}/csw'
csw = CatalogueServiceWeb(system_catalogue_endpoint, timeout=30)

In [26]:
print(f"CSW GetCapabilities: {csw.identification.title}")
print(f"CSW Version: {csw.identification.version}")
print(f"CSW Service Type: {csw.identification.type}")
print(f"CSW Operations: {[op.name for op in csw.operations]}")


CSW GetCapabilities: EOEPCA+ Resource Catalogue
CSW Version: 2.0.2
CSW Service Type: CSW
CSW Operations: ['GetCapabilities', 'DescribeRecord', 'GetDomain', 'GetRecords', 'GetRecordById', 'GetRepositoryItem', 'Transaction', 'Harvest']


In [27]:
csw.getrecords2(maxrecords=10)
csw.results

for rec in csw.records:
   print(f'identifier: {csw.records[rec].identifier}\ntype: {csw.records[rec].type}\ntitle: {csw.records[rec].title}\n')

identifier: example-item
type: item
title: None

identifier: example-item-2
type: item
title: None

identifier: example-item-3
type: item
title: None



In [30]:
bbox_query = BBox(
    [
        -180.0,
        -90.0,
        180.0,
        90.0
    ]
)

begin = PropertyIsGreaterThanOrEqualTo(propertyname='apiso:TempExtent_begin', literal='2012-09-10 00:00')

filter_list = [
    And(
        [
            bbox_query,
            begin
        ]
    )
]

csw.getrecords2(constraints=filter_list, outputschema='http://www.isotc211.org/2005/gmd')
csw.results


{'matches': 3, 'returned': 3, 'nextrecord': 0}

In [32]:
collection_query = PropertyIsEqualTo('apiso:ParentIdentifier', 'metadata:main')
csw.getrecords2(constraints=[collection_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

{'matches': 3, 'returned': 3, 'nextrecord': 0}

In [33]:
csw.getrecords2(constraints=[bbox_query], outputschema='http://www.isotc211.org/2005/gmd')
csw.results

{'matches': 3, 'returned': 3, 'nextrecord': 0}

# OpenSearch

In [35]:
opensearch_endpoint = f'{resource_discovery_domain}/csw?service=CSW&version=3.0.0&request=GetCapabilities&mode=opensearch'
os = OpenSearch(opensearch_endpoint)

In [36]:
print(f"Short name: {os.description.shortname}")
print(f"Long name: {os.description.longname}")
print(f"Description: {os.description.description}")
print(f"URLs: {os.description.urls}")

Short name: EOEPCA+ Resource
Long name: EOEPCA+ Resource Catalogue
Description: EOEPCA+ Resource Catalogue
URLs: {'application/xml': {'rel': None, 'template': 'https://resource-catalogue.test.eoepca.org/csw?service=CSW&version=3.0.0&request=GetRecords&elementsetname=full&typenames=csw:Record&outputformat=application/xml&outputschema=http://www.opengis.net/cat/csw/3.0&recordids={geo:uid?}&q={searchTerms?}&bbox={geo:box?}&time={time:start?}/{time:end?}&start={time:start?}&stop={time:end?}&startposition={startIndex?}&maxrecords={count?}&eo:cloudCover={eo:cloudCover?}&eo:instrument={eo:instrument?}&eo:orbitDirection={eo:orbitDirection?}&eo:orbitNumber={eo:orbitNumber?}&eo:parentIdentifier={eo:parentIdentifier?}&eo:platform={eo:platform?}&eo:processingLevel={eo:processingLevel?}&eo:productType={eo:productType?}&eo:sensorType={eo:sensorType?}&eo:snowCover={eo:snowCover?}&eo:spectralRange={eo:spectralRange?}', 'parameters': {}}, 'application/atom+xml': {'rel': None, 'template': 'https://resou

In [37]:
results = os.search('application/atom+xml')
len(results)

unknown link type in Ows Context section: {'href': 'https://resource-catalogue.test.eoepca.org/csw?mode=opensearch&service=CSW&version=3.0.0&request=GetCapabilities', 'type': 'application/opensearchdescription+xml', 'length': None, 'lang': None, 'title': None, 'rel': 'search'}
unknown link type in Ows Resource entry section: {'href': 'https://resource-catalogue.test.eoepca.org/csw?service=CSW&version=2.0.2&request=GetRepositoryItem&id=example-item', 'type': None, 'length': None, 'lang': None, 'title': None, 'rel': None}
unknown link type in Ows Resource entry section: {'href': 'https://resource-catalogue.test.eoepca.org/csw?service=CSW&version=2.0.2&request=GetRepositoryItem&id=example-item-2', 'type': None, 'length': None, 'lang': None, 'title': None, 'rel': None}
unknown link type in Ows Resource entry section: {'href': 'https://resource-catalogue.test.eoepca.org/csw?service=CSW&version=2.0.2&request=GetRepositoryItem&id=example-item-3', 'type': None, 'length': None, 'lang': None, 't

5

# OGC API Records

In [38]:
w = Records(resource_discovery_domain)

In [39]:
print(f"URLs: {w.url}")
print(f"Conformance: {w.conformance()}")
print(f"API Version: {w.api()}")

URLs: https://resource-catalogue.test.eoepca.org/
Conformance: {'conformsTo': ['http://www.opengis.net/spec/ogcapi-common-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-common-2/1.0/conf/collections', 'http://www.opengis.net/spec/ogcapi-features-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/queryables', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/queryables-query-parameters', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/filter', 'http://www.opengis.net/spec/ogcapi-features-3/1.0/conf/features-filter', 'http://www.opengis.net/spec/ogcapi-features-4/1.0/conf/create-replace-delete', 'http://www.opengis.net/spec/ogcapi-records-1/1.0/conf/core', 'http://www.opengis.net/spec/ogcapi-records-1/1.0/conf/sorting', 'http://www.opengis.net/spec/ogcapi-records-1/1.0/conf/json', 'http://www.opengis.net/spec/ogcapi-records-1/1.0/conf/html', 'http://www.opengis.net/spec/cql2/1.0/conf/cql2-json', 'http://www.opengis.net/spec/cql2/1.0/conf/cql2-t

In [40]:
records = w.records()
print(f"Records: {records}")

Records: ['metadata:main']


In [41]:
my_catalogue = w.collection('metadata:main')
print(f"Catalogue: {my_catalogue['id']}")
print(f"Queryable: {w.collection_queryables('metadata:main')}")

Catalogue: metadata:main
Queryable: {'id': 'metadata:main', 'type': 'object', 'title': 'EOEPCA+ Resource Catalogue', 'properties': {'geometry': {'$ref': 'https://geojson.org/schema/Polygon.json', 'x-ogc-role': 'primary-geometry'}, 'identifier': {'title': 'identifier', 'x-ogc-role': 'id', 'type': 'string'}, 'typename': {'title': 'typename', 'type': 'string'}, 'type': {'title': 'type', 'type': 'string'}, 'title': {'title': 'title', 'type': 'string'}, 'edition': {'title': 'edition', 'type': 'string'}, 'keywords': {'title': 'keywords', 'type': 'string'}, 'parentidentifier': {'title': 'parentidentifier', 'type': 'string'}, 'time_begin': {'title': 'time_begin', 'type': 'string'}, 'time_end': {'title': 'time_end', 'type': 'string'}, 'date': {'title': 'date', 'type': 'string'}, 'platform': {'title': 'platform', 'type': 'string'}, 'instrument': {'title': 'instrument', 'type': 'string'}, 'sensortype': {'title': 'sensortype', 'type': 'string'}}, '$schema': 'http://json-schema.org/draft/2019-09/sc

In [42]:
my_catalogue_query = w.collection_items('metadata:main')
print(f"Matched records: {my_catalogue_query['numberMatched']}")
print(f"Metadata of first result: {my_catalogue_query['features'][0]['properties']}")


Matched records: 3
Metadata of first result: {'datetime': '2024-01-01T00:00:00Z'}


In [43]:
spatial_query = w.collection_items('metadata:main', bbox=[-180.0, -90.0, 180.0, 90.0])
print(f"Matched records: {spatial_query['numberMatched']}")
print([f['id'] for f in spatial_query['features']])

Matched records: 3
['example-item', 'example-item-2', 'example-item-3']


In [67]:
my_catalogue_datetime_query = w.collection_items(
    'metadata:main',
    cql={
        'op': '>',
        'args': [
            {'property': 'datetime'},
            '2026-01-02T00:00:00Z'
        ]
    }
)

print("Matched items:", my_catalogue_datetime_query['numberMatched'])
print("Item IDs:", [f['id'] for f in my_catalogue_datetime_query['features']])

Matched items: 3
Item IDs: ['example-item', 'example-item-2', 'example-item-3']


## STAC Items Search

In [68]:
search_url = f"{resource_discovery_domain}/stac/search"
search_payload = {
    "bbox": [-180.0, -90.0, 180.0, 90.0],
    "limit": 3
}

response = requests.post(search_url, json=search_payload)
print(f"STAC search status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

STAC search status code: 200
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "stac_version": "1.0.0",
      "id": "example-item",
      "properties": {
        "datetime": "2024-01-01T00:00:00Z"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          0.0,
          0.0
        ]
      },
      "bbox": [
        0.0,
        0.0,
        0.0,
        0.0
      ],
      "assets": {
        "example-data": {
          "href": "http://example.com/data.tif",
          "type": "image/tiff"
        }
      },
      "links": [
        {
          "rel": "self",
          "type": "application/geo+json",
          "href": "https://resource-catalogue.test.eoepca.org/stac/collections/metadata:main/items/example-item"
        },
        {
          "rel": "root",
          "type": "application/json",
          "href": "https://resource-catalogue.test.eoepca.org/stac"
        },
        {
          "rel": "parent",
          "typ